# Group Details

## Group Name: 12

### Student 1: Denise La Gordt Dillie

### Student 2: Andreea Maican

### Student 3: Sambhav Jain

# Loading Data and Preliminaries

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import torch
from torch_geometric.data import Data
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import os

In [2]:
def load_array(filename, task):
    datapoint = np.load(filename)
    if task == 'task 1':
        initial_state = datapoint['initial_state']
        terminal_state = datapoint['terminal_state']
        return initial_state, terminal_state
    elif task == 'task 2' or task == 'task 3':
        whole_trajectory = datapoint['trajectory']
        # change shape: (num_bodies, attributes, time) ->  num_bodies, time, attributes
        whole_trajectory = np.swapaxes(whole_trajectory, 1, 2)
        initial_state = whole_trajectory[:, 0]
        target = whole_trajectory[:, 1:, 1:]  # drop the first timepoint (second dim) and mass (last dim) for the prediction task
        return initial_state, target
    else:
        raise NotImplementedError("'task' argument should be 'task 1', 'task 2' or 'task 3'!")


In [3]:
"""
This cell gives an example of loading a datapoint with numpy for task 1.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
terminal_state: shape (n_bodies, [x, y])

"""

example = load_array('C:/University/master/Q4/Deep Learning/data-assignment2/data/task 1/train/trajectory_0.npz', task='task 1')

initial_state, terminal_state = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(initial_state)
print(f'shape of terminal state (to be predicted by model): {terminal_state.shape}')

body_idx = 2
print(f'The initial x-coordinate of the body with index {body_idx} in this trajectory was {initial_state[body_idx, 1]}')

shape of initial state (model input): (8, 5)
[[ 4.54673709 -5.27118739  5.07863417 -1.09690628 -3.66929964]
 [ 3.57509525  4.3270607  -0.08095022 -0.57868726 -2.95971243]
 [ 2.67733735 -5.15972108  5.35238208  2.42652043  1.45870728]
 [ 2.27455417 -6.79584511  2.29632123  0.30418238 -0.4065998 ]
 [ 2.38359341 -2.75861066  1.77940931 -0.07643627 -0.35954359]
 [ 4.28596268  0.34981219  4.8286224   1.35630962 -2.55760522]
 [ 3.03118516 -0.50562258 15.04631712 -0.94772523 14.98127867]
 [ 4.13530017  1.81342682  3.84375499 -0.6989711  -2.25092411]]
shape of terminal state (to be predicted by model): (8, 2)
The initial x-coordinate of the body with index 2 in this trajectory was -5.159721083543527


In [4]:
"""
This cell gives an example of loading a datapoint with numpy for task 2 / 3.

The arrays returned by the function are structures as follows:
initial_state: shape (n_bodies, [mass, x, y, v_x, v_y])
remaining_trajectory: shape (n_bodies, time, [x, y, v_x, v_y])

Note that for this task, you are asked to evaluate performance only with regard to the predictions of the positions (x and y).
If you use the velocity of the remaining trajectory for training,
this use should be purely auxiliary for the goal of predicting the positions [x,y] over time. 
While testing performance of your model on the test set, you do not have access to v_x and v_y of the remaining trajectory.

"""

example = load_array('C:/University/master/Q4/Deep Learning/data-assignment2/data/task 2_3/train/trajectory_0.npz', task='task 2')

initial_state, remaining_trajectory = example
print(f'shape of initial state (model input): {initial_state.shape}')
print(f'shape of terminal state (to be predicted by model): {remaining_trajectory.shape}')

body_idx = 2
time_idx = 30
print(f'The y-coordinate of the body with index {body_idx} at time with index {time_idx} in remaining_trajectory was {remaining_trajectory[body_idx, time_idx, 1]}')

test_example = load_array('C:/University/master/Q4/Deep Learning/data-assignment2/data/task 2_3/test/trajectory_900.npz', task='task 3')
test_initial_state, test_remaining_trajectory = test_example
print(f'the shape of the input of a test data example is {test_initial_state.shape}')
print(f'the shape of the target of a test data example is {test_remaining_trajectory.shape}')
print(f'values of the test data example at time {time_idx}:\n {test_remaining_trajectory[:, time_idx]}')
print('note: velocity values are unobserved (NaNs) in the test data!')

shape of initial state (model input): (8, 5)
shape of terminal state (to be predicted by model): (8, 49, 4)
The y-coordinate of the body with index 2 at time with index 30 in remaining_trajectory was -0.3861544940435097
the shape of the input of a test data example is (8, 5)
the shape of the target of a test data example is (8, 49, 4)
values of the test data example at time 30:
 [[-5.85725792 -5.394571           nan         nan]
 [-6.03781257 -5.72445953         nan         nan]
 [-0.90623054 -6.93416278         nan         nan]
 [ 2.83149339 -7.50100819         nan         nan]
 [-2.85586881  1.77667501         nan         nan]
 [ 4.04424526  4.00563603         nan         nan]
 [-5.24887713 -4.83081005         nan         nan]
 [-5.81391023 -5.1109838          nan         nan]]
note: velocity values are unobserved (NaNs) in the test data!


In [5]:
test_initial_state

array([[ 4.31455329, -4.99311363, -5.10150489,  2.2491171 , -1.82520359],
       [ 2.58525749, -4.92574075, -4.7534334 , -4.81897728, -3.07212095],
       [ 2.40449047,  2.23254187, -6.04595524, -0.67381217, -0.58798851],
       [ 2.28318603,  5.6605113 , -3.76376281, -0.41268251, -1.19812499],
       [ 4.09793595, -4.73678906, -3.94704856,  1.13634781,  4.98383634],
       [ 4.5579536 , -0.08848969,  5.42483477,  1.460317  , -0.21585929],
       [ 3.42356878, -4.91282501, -4.6779343 ,  0.94123905,  2.66425652],
       [ 3.59288181, -4.48225046, -4.548729  , -2.56573061, -2.39203918]])

# Data Handling and Preprocessing

In [78]:
def create_graph_data(initial_state, terminal_state):
    '''Takes a datapoint from task 1 data and returns a Data object ready to be used with
    Pytorch Geometric. Each node is an object and will store initial position and speed, each edge stores
    the distance between two objects and also their masses.'''
    initial_state = np.array(initial_state)
    terminal_state = np.array(terminal_state)

    n_bodies = initial_state.shape[0]
    num_features = initial_state.shape[1]

    # Create node features tensor
    node_features = torch.tensor(initial_state, dtype=torch.float)
    node_features = node_features.view(-1, num_features)  # Reshape to (num_nodes, num_features)

    # Create edge index tensor
    edge_index = torch.tensor([[i, j] for i in range(n_bodies) for j in range(n_bodies) if (i != j and i<j)], dtype=torch.long)
    edge_index = edge_index.t().contiguous()  # Reshape to (2, num_edges)

    # Create edge features tensor
    num_edges = edge_index.size(1)
    edge_features = torch.empty(num_edges, 1, dtype=torch.float) 
    # Populate edge features with force
    for k, (i, j) in enumerate(edge_index.t()):
        dx = node_features[i, 1] - node_features[j, 1]
        dy = node_features[i, 2] - node_features[j, 2]
        distance = torch.sqrt(dx * 2 + dy * 2)
        force = ((node_features[i,0]*node_features[j,0])/(distance*2))
        edge_features[k] = force

    # Create target tensor
    target = torch.tensor(terminal_state, dtype=torch.float).view(-1, 2)  # Reshape to (num_nodes, num_features + 1)

    # Create PyG Data object
    data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features, y=target)

    return data

# Example usage
initial_state = [[1.0, 0.5, 0.3, 0.2, 0.1], [2.0, 0.1, 0.4, 0.3, 0.2], [0.5, 0.7, 0.8, 0.9, 1.0]]
terminal_state = [[0.3, 0.2], [0.4, 0.6], [0.8, 0.9]]

graph_data = create_graph_data(initial_state, terminal_state)
print(graph_data.x, graph_data.edge_index, graph_data.edge_attr, graph_data.y)

tensor([[1.0000, 0.5000, 0.3000, 0.2000, 0.1000],
        [2.0000, 0.1000, 0.4000, 0.3000, 0.2000],
        [0.5000, 0.7000, 0.8000, 0.9000, 1.0000]]) tensor([[0, 0, 1],
        [1, 2, 2]]) tensor([[1.2910],
        [   nan],
        [   nan]]) tensor([[0.3000, 0.2000],
        [0.4000, 0.6000],
        [0.8000, 0.9000]])


In [79]:
import random

def create_dataloader(data_folder, task, shuffle, batch_size, train_ratio=0.8):
    data_list = []  # List to store the Data objects

    # Loop over all files in the folder
    for filename in os.listdir(data_folder):
        file_path = os.path.join(data_folder, filename)

        # Read the data from the file (modify this part based on your file format)
        states = load_array(file_path, task=task)
        initial_state, terminal_state = states

        # Create the Data object using create_graph_data function
        data = create_graph_data(initial_state, terminal_state)

        # Append the Data object to the list
        data_list.append(data)

    # Shuffle the data if specified
    if shuffle:
        random.shuffle(data_list)

    # Calculate the number of samples
    num_samples = len(data_list)

    if "train" in data_folder:
        # Calculate the number of samples for train and validation
        num_train = int(train_ratio * num_samples)
        num_val = num_samples - num_train

        # Split the data into train and validation sets
        train_data = data_list[:num_train]
        val_data = data_list[num_train:]

        # Print the number of data samples
        print(f"Number of data samples in {data_folder}: {num_samples}")
        print(f"Number of training samples: {num_train}")
        print(f"Number of validation samples: {num_val}")

        # Create data loaders for train and validation sets
        dataloader_train = DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
        dataloader_val = DataLoader(val_data, batch_size=batch_size, shuffle=False)

        return dataloader_train, dataloader_val
    else:
        # Print the number of data samples
        print(f"Number of data samples in {data_folder}: {num_samples}")

        # Create a data loader for the data
        dataloader = DataLoader(data_list, batch_size=batch_size, shuffle=shuffle)

        return dataloader

# Specify the paths to your train and test data folders
data_folder_train = 'C:/University/master/Q4/Deep Learning/data-assignment2/data/task 1/train'
data_folder_test = 'C:/University/master/Q4/Deep Learning/data-assignment2/data/task 1/test'

# Define the batch size
batch_size = 50

# Create data loaders for train and validation sets (only if the folder contains "train")
dataloader_train, dataloader_val = create_dataloader(data_folder_train, task='task 1', shuffle=True, batch_size=batch_size)

# Create a separate data loader for the test set
dataloader_test= create_dataloader(data_folder_test, task='task 1', shuffle=False, batch_size=batch_size)

Number of data samples in C:/University/master/Q4/Deep Learning/data-assignment2/data/task 1/train: 900
Number of training samples: 720
Number of validation samples: 180
Number of data samples in C:/University/master/Q4/Deep Learning/data-assignment2/data/task 1/test: 100


# Model Implementation

# Model Training

In [80]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device='cpu'
print(f'Loaded device: {device}')

Loaded device: cpu


In [83]:

import math
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


# class GNN(torch.nn.Module):
#     def __init__(self, hidden_dim=64):
#         super(GNN, self).__init__()
#         self.conv1 = GCNConv(4, hidden_dim) 
#         self.conv2 = GCNConv(hidden_dim, hidden_dim)
#         # self.lstm = torch.nn.LSTM(hidden_dim, hidden_dim)
#         self.lin = torch.nn.Linear(hidden_dim, 2) 

#     def forward(self, data):
#         x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
#         h = F.relu(self.conv1(x, edge_index, edge_attr))
#         h = F.relu(self.conv2(h, edge_index, edge_attr))
#         # h = h.unsqueeze(0)
#         # _, (h_n, _) = self.lstm(h)
#         out = self.lin(h)
#         return out


class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(5, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.conv4 = GCNConv(256, 128)
        self.conv5 = GCNConv(128, 2)

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)

        return x



# model = GNNModel(num_features=4, hidden_dim=64, num_classes=2).to(device)

model = GNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss_func = torch.nn.MSELoss()

def train():
    model.train()
    for data in dataloader_train:
        data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_func(out, data.y)
        loss.backward()
        optimizer.step()
        
        
def test(loader):
    model.eval()
    mse = 0.0
    total = 0

    for data in loader:
        data.to(device)
        out = model(data)
        if data.y is not None:  # Skip calculation if data.y is None
            mse += F.mse_loss(out, data.y, reduction='sum').item()
            total += data.y.size(0)

    if total == 0:
        return None  # Return None if no samples with valid labels are found

    mse /= total
    rmse = math.sqrt(mse)

    return rmse


train_losses = []
test_losses = []
val_losses = []

epochs = 50
start = time.time()

for epoch in range(1, epochs + 1):
    train()
    with torch.no_grad():
        train_loss = test(dataloader_train)
        test_loss = test(dataloader_test)
        val_loss = test(dataloader_val)
    toc = time.time()
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Val Loss: {val_loss:.4f} . Training time so far: {toc - start:.1f} s')
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    val_losses.append(val_loss)




Epoch: 001, Train Loss: 6.8185, Test Loss: 6.4203, Val Loss: 6.4649 . Training time so far: 0.6 s
Epoch: 002, Train Loss: 6.7914, Test Loss: 6.4064, Val Loss: 6.4254 . Training time so far: 1.1 s
Epoch: 003, Train Loss: 6.7828, Test Loss: 6.4079, Val Loss: 6.4163 . Training time so far: 1.5 s
Epoch: 004, Train Loss: 6.7766, Test Loss: 6.4000, Val Loss: 6.4050 . Training time so far: 1.9 s
Epoch: 005, Train Loss: 6.7726, Test Loss: 6.3961, Val Loss: 6.4021 . Training time so far: 2.3 s
Epoch: 006, Train Loss: 6.7689, Test Loss: 6.3918, Val Loss: 6.3971 . Training time so far: 2.7 s
Epoch: 007, Train Loss: 6.7651, Test Loss: 6.3885, Val Loss: 6.3966 . Training time so far: 3.2 s
Epoch: 008, Train Loss: 6.7616, Test Loss: 6.3896, Val Loss: 6.3959 . Training time so far: 3.6 s
Epoch: 009, Train Loss: 6.7580, Test Loss: 6.3838, Val Loss: 6.3889 . Training time so far: 4.0 s
Epoch: 010, Train Loss: 6.7549, Test Loss: 6.3845, Val Loss: 6.3886 . Training time so far: 4.4 s
Epoch: 011, Train Lo

In [96]:
# import time
# import torch.nn.functional as F
# import math


# model = MpGNN(node_feat_dim=4, hidden_dim=32, num_layers=10)
# model.to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr= 0.009)
# loss_func = torch.nn.MSELoss()

# def train():
#     model.train()
#     for data in dataloader_train:
#         data.to(device)
#         optimizer.zero_grad()
#         out = model(data.x, data.edge_index, data.batch)
#         loss = loss_func(out, data.y)
#         loss.backward()
#         optimizer.step()
        
        
# def test(loader):
#     model.eval()
#     mse = 0.0
#     total = 0

#     for data in loader:
#         data.to(device)
#         out = model(data.x, data.edge_index, data.batch)
#         mse += F.mse_loss(out, data.y, reduction='sum').item()
#         total += data.y.size(0)

#     mse /= total
#     rmse = math.sqrt(mse)

#     return rmse

# train_losses = []
# test_losses = []
# val_losses = []

# epochs = 50
# start = time.time()

# for epoch in range(1, epochs + 1):
#     train()
#     with torch.no_grad():
#         train_loss = test(dataloader_train)
#         test_loss = test(dataloader_test)
#         val_loss = test(dataloader_val)
#     toc = time.time()
#     print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Val Loss: {val_loss:.4f} . Training time so far: {toc - start:.1f} s')
#     train_losses.append(train_loss)
#     test_losses.append(test_loss)
#     val_losses.append(val_loss)

Epoch: 001, Train Loss: 6.7892, Test Loss: 6.4481, Val Loss: 6.7635 . Training time so far: 0.5 s
Epoch: 002, Train Loss: 6.7859, Test Loss: 6.4445, Val Loss: 6.7599 . Training time so far: 0.9 s
Epoch: 003, Train Loss: 6.7843, Test Loss: 6.4428, Val Loss: 6.7596 . Training time so far: 1.4 s
Epoch: 004, Train Loss: 6.7857, Test Loss: 6.4371, Val Loss: 6.7574 . Training time so far: 1.9 s
Epoch: 005, Train Loss: 6.7842, Test Loss: 6.4360, Val Loss: 6.7575 . Training time so far: 2.4 s
Epoch: 006, Train Loss: 6.7838, Test Loss: 6.4416, Val Loss: 6.7604 . Training time so far: 2.8 s
Epoch: 007, Train Loss: 6.7852, Test Loss: 6.4408, Val Loss: 6.7596 . Training time so far: 3.3 s
Epoch: 008, Train Loss: 6.7835, Test Loss: 6.4365, Val Loss: 6.7591 . Training time so far: 3.7 s
Epoch: 009, Train Loss: 6.7853, Test Loss: 6.4377, Val Loss: 6.7584 . Training time so far: 4.1 s
Epoch: 010, Train Loss: 6.7857, Test Loss: 6.4411, Val Loss: 6.7595 . Training time so far: 4.6 s
Epoch: 011, Train Lo

In [ ]:
def plot_train(train_losses, test_losses, val_losses):
    fig, ax = plt.subplots(figsize=(8, 6))
    fnt = 16
    epochs = range(1, len(train_losses) + 1)
    ax.plot(epochs, train_losses, color='blue', label='Train')
    ax.plot(epochs, test_losses, color='green', linestyle='--', label='Test')
    ax.plot(epochs, val_losses, color='red', linestyle='-.', label='Validation')
    ax.legend(fontsize=fnt)
    ax.tick_params(axis='both', labelsize=fnt)
    ax.set_xlabel('Epoch', fontsize=fnt)
    ax.set_ylabel('Loss', fontsize=fnt)


In [ ]:
plot_train(train_losses, test_losses, val_losses)

# Evaluation

In [ ]:
print(f'Final Test accuracy: {train_losses[-1]}')